# Clean dataset

In the previous notebook, we used Web Scraping to collect information from Wikipedia webpages and combine them together in a .CSV file named 'Dataset'. However, this dataset is not available in the most useful format and must be cleaned before any analysis could be performed. Such a scenario is expected to occur as the web pages aren't always designed to make extracting data easy.

Here, we'll understand the dataset columns and try to remove any unnecessary strings and values that bring no value to the dataset.

## Import libraries and read CSV file

We'll first import the Pandas library which enables us to manipulate and work with the dataset. Then, we'll simply read the dataset file we created.

In [ ]:
import re
import numpy as np
import pandas as pd

dataset = pd.read_csv("Dataset.csv")
dataset.head(5)

## Replace the headings

We first begin by replacing the headings of the columns such that they reflect the data in them better. We change the text for the first column to 'Country', the third column to 'Percentage of World Population' and fourth column to 'Total Area (km2)'.

In [ ]:
dataset.rename(columns={'Country(or dependent territory)': 'Country'}, inplace = True)
dataset.rename(columns={'% of worldpopulation': 'Percentage of World Population'}, inplace = True)
dataset.rename(columns={'Total Area': 'Total Area (km2)'}, inplace = True)
dataset.head(5)

## Analysing the dataset

We see that almost all columns have cells which have data inside parentheses and square brackets which is not required. Thus, we can first remove all paranthesis, square brackets and the content inside them.

In [ ]:
for column in dataset.columns:
    dataset[column] = dataset[column].str.replace(r"\(.*\)", "")
    dataset[column] = dataset[column].str.replace(r"\[.*\]", "")

dataset.head(5)

Next, we do not need '%' sign in either column 3 or 5, thus, we can strip the cells of it.

In [ ]:
dataset['Percentage of World Population'] = dataset['Percentage of World Population'].str.strip('%')
dataset['Percentage Water'] = dataset['Percentage Water'].str.strip('%')
dataset['Percentage Water'] = dataset['Percentage Water'].str.strip()

dataset.sample(5)

Next, we remove commas from Population column.

In [ ]:
dataset['Population'] = dataset['Population'].str.replace(',', '')

dataset.head(10)

Now, we will explore the area column. Initially, we see that the information is represented in two units: sq mi and km2. We need to convert all values to km2.

The formula to convert 'sq mi' to km2 is to multiply the value by 2.58999.

First, we check if the cell has the units as 'sq mi', then we multiply it with 2.589999, convert it to integer and save it back to the cell else we simply convert it into integer. Before this, on taking a closer look at the values, some cells have range of areas and as a result we need to split the data at '-' and then take the first value to continue further.

In [ ]:
dataset['Total Area (km2)'] = dataset['Total Area (km2)'].str.replace(',', '')

for x in range(len(dataset['Total Area (km2)'])):
    area = dataset.iloc[x]['Total Area (km2)']
    if ('sq\xa0mi' in area):
        area = area.split('-')[0]
        area = re.sub(r'[^0-9.]+', '', area)
        area = int(float(area) * 2.58999)
    else:
        area = area.split('-')[0]
        area = re.sub(r'[^0-9.]+', '', area)
        area = int(float(area))
    dataset.iloc[x]['Total Area (km2)'] = area

dataset.head(5)

Let's analyse the 'Percentage Water' column further. 
For Algeria, Afghanistan, and some other countries, the value is negligible. Hence, in order to retain data and not drop these rows, we will mark these cells with 0.0.
Chile has the character 'b' in the end which needs to be removed.
For the columns where the value is more than 100, the actual values were missing and other content has been read instead. Thus, we must remove such rows due to lack of information.

In [ ]:
dataset['Percentage Water'] = dataset['Percentage Water'].replace('negligible', '0.0')
dataset['Percentage Water'] = dataset['Percentage Water'].replace('Negligible', '0.0')
dataset['Percentage Water'] = dataset['Percentage Water'].str.replace(r'[^0-9.]+', '')

dataset = dataset[dataset['Percentage Water'].astype(float) <= 100]

dataset.head(5)

Total GDP includes the values in the form of trillions, billions and millions. We can remove '$' and convert the words to numbers.

In [ ]:
dataset['Total Nominal GDP'] = dataset['Total Nominal GDP'].str.replace('$', '')

for x in range(len(dataset['Total Nominal GDP'])):
    gdp = dataset.iloc[x]['Total Nominal GDP']
    if ('trillion' in dataset.iloc[x]['Total Nominal GDP']):
        gdp = re.sub(r'[^0-9.]+', '', gdp)
        gdp = int(float(gdp) * 1000000000000)
    elif ('billion' in dataset.iloc[x]['Total Nominal GDP']):
        gdp = re.sub(r'[^0-9.]+', '', gdp)
        gdp = int(float(gdp) * 1000000000)
    elif ('million' in dataset.iloc[x]['Total Nominal GDP']):
        gdp = re.sub(r'[^0-9.]+', '', gdp)
        gdp = int(float(gdp) * 1000000)
    else:
        gdp = int(re.sub(r'[^0-9.]+', '', gdp))
    dataset.iloc[x]['Total Nominal GDP'] = gdp

We can remove the '$' sign as well as comma from the Per Capita GDP as well.

In [ ]:
dataset['Per Capita GDP'] = dataset['Per Capita GDP'].str.replace(r'[^0-9.]+', '')

dataset.head(10)

In [ ]:
dataset.to_csv("Final_dataset.csv", index = False)

## Conclusion

We have iterated through all columns of the dataset and removed unnecesaary characters, and unified all data into a common format. Each column presented its own set of difficulties that had to be tackled with.
The final dataset is ready which we can use for further analysis.